# MGMTMSA 403 Homework 2: Portfolio Optimization

## Pre-processing step: Estimate expected returns and covariance

In [1]:
# Import gurobi and numpy
from gurobipy import *
import pandas as pd
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0] # 61
n = prices.shape[1] # 390

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))

## Question 1

### a) Model 1

Find monthly and average returns for MSFT, GS, PG, and SCHP below:

In [2]:
df = pd.DataFrame(data = returns)
returns_mod_1 = df[[315, 216, 372, 388]]
returns_mod_1 = returns_mod_1.rename(columns={315: 'MSFT', 216: 'GS', 372: 'PG', 388: 'SCHP'})

returns_mod_1.head()

,MSFT,GS,PG,SCHP
0,0.016129,-0.073446,-0.048447,0.024668
1,0.000000,0.006098,0.011749,0.009259
2,0.084656,-0.006061,0.040000,-0.009174
3,-0.012195,-0.036585,0.022333,0.009259
4,-0.012346,0.018987,-0.027913,0.023853


In [3]:
avg_return_mod_1 = np.zeros(4)
avg_return_mod_1 = np.mean(returns_mod_1,axis=0)

avg_return_mod_1

MSFT    0.019688
GS      0.005784
PG      0.001068
SCHP    0.000247
dtype: float64

Find covariance matrix for MSFT, GS, PG, and SCHP below:

In [4]:
C_mod_1 = np.zeros((4,4))
C_mod_1 = np.cov(np.transpose(returns_mod_1))

cov_mod_1 = pd.DataFrame(data = C_mod_1)
cov_mod_1 = cov_mod_1.rename(index={0: 'MSFT', 1: 'GS', 2: 'PG', 3: 'SCHP'}, columns={0: 'MSFT', 1: 'GS', 2: 'PG', 3: 'SCHP'})

cov_mod_1

,MSFT,GS,PG,SCHP
MSFT,0.002732,0.001172,0.000625,-0.000139
GS,0.001172,0.003727,-0.000013,-0.000238
PG,0.000625,-0.000013,0.001357,0.000107
SCHP,-0.000139,-0.000238,0.000107,0.000119


Create optimization model below:

In [5]:
# Define model. 
mod_1 = Model()

# Define decision variables.
w_msft = mod_1.addVar()
w_gs = mod_1.addVar()
w_pg = mod_1.addVar()
w_schp = mod_1.addVar()

w_mod_1 = [w_msft, w_gs, w_pg, w_schp] # array of weights

# Constraint 1 - expected monthly return
return_con_mod_1 = mod_1.addConstr((.019688)*w_msft + (.005784)*w_gs + (.001068)*w_pg + (.000247)*w_schp  >= 0.005)

# Constraint 2 - sum of weights
weight_con_mod_1 = mod_1.addConstr(w_msft + w_gs + w_pg + w_schp == 1)

# Constraint 3 - non-negativity constraints
mod_1.addConstr(w_msft >= 0.0)
mod_1.addConstr(w_gs >= 0.0)
mod_1.addConstr(w_pg >= 0.0)
mod_1.addConstr(w_schp >= 0.0)

# Create the objective function, and set it to be minimized.
mod_1.setObjective(sum(w_mod_1[i] * w_mod_1[j] * C_mod_1[i, j] for i in range(4) for j in range(4)), GRB.MINIMIZE)

mod_1.update()

mod_1.optimize()

# Get optimal weights for each of the assets
w_opt_mod_1 = [w_mod_1[i].x for i in range(4)]
print(w_opt_mod_1)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-10
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x8b3fafe1
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time


**Optimal risk:** 1.77493968e-04 <br>
**Solver time:** 10 iterations and 0.02 seconds <br>
**Weights:** MSFT = 0.23712, GS = 0.02586, PG = 1.57557e-09, SCHP = 0.73702

### b) Model 2

Find monthly and average returns for all 390 stocks below:

In [6]:
returns_mod_2 = pd.DataFrame(data = returns)

returns_mod_2.head()

,0,1,2,3,4,5,6,7,8,9,...,380,381,382,383,384,385,386,387,388,389
0,-0.072464,-0.039267,-0.053846,0.022305,-0.014797,-0.055249,-0.001686,-0.131646,0.045455,-0.013986,...,-0.003195,0.012278,-0.005682,-0.102857,-0.011650,0.070946,0.093834,0.045652,0.024668,0.014981
1,0.031250,0.070845,0.018293,0.007273,0.032541,0.140351,0.146959,0.069971,0.095652,-0.014184,...,0.134615,-0.008086,-0.001905,0.019108,0.023576,0.032597,0.078431,0.051975,0.009259,0.002460
2,0.030303,-0.020356,0.025948,-0.061372,-0.033939,0.046154,-0.032401,0.092643,0.007937,0.028777,...,0.042373,0.072011,0.104962,0.006250,0.072937,0.006110,-0.070455,0.009881,-0.009174,-0.003681
3,0.014706,0.000000,0.000000,-0.076923,0.000000,-0.034314,-0.073059,0.004988,-0.055118,0.000000,...,-0.059621,-0.005070,-0.037997,-0.055901,-0.118068,-0.029352,0.061125,0.027397,0.009259,0.003695
4,0.021739,0.031169,0.052529,-0.095833,-0.001255,0.030457,0.064039,-0.007444,0.008333,-0.027972,...,-0.017291,0.021656,0.043088,0.006579,-0.014199,0.007299,-0.004608,0.015238,0.023853,0.012270


In [7]:
avg_return_mod_2 = np.zeros(n)
avg_return_mod_2 = np.mean(returns_mod_2,axis=0)

avg_return_mod_2

0      0.008356
1      0.011798
2      0.012202
3      0.053553
4      0.013364
         ...   
385    0.015104
386    0.008576
387    0.006485
388    0.000247
389   -0.000476
Length: 390, dtype: float64

Find covariance matrix for all 390 stocks below:

In [8]:
C_mod_2 = np.zeros((n,n))
C_mod_2 = np.cov(np.transpose(returns_mod_2))

cov_mod_2 = pd.DataFrame(data = C_mod_2)

cov_mod_2

,0,1,2,3,4,5,6,7,8,9,...,380,381,382,383,384,385,386,387,388,389
0,0.002124,0.001198,0.001128,0.001055,0.000955,0.001135,0.000895,0.002331,0.000414,0.000843,...,0.000710,0.001010,0.001060,0.000791,0.001240,0.000721,0.000239,0.000470,-0.000028,-0.000021
1,0.001198,0.002620,0.001678,0.001594,0.000780,0.001181,0.001134,0.003082,0.000371,0.000887,...,0.000461,0.000692,0.001065,0.000841,0.001935,0.000503,0.000168,0.000167,-0.000076,-0.000060
2,0.001128,0.001678,0.004387,0.002304,0.000666,0.000355,0.000533,0.000991,-0.000377,0.000162,...,0.000577,0.000524,0.000629,0.001823,0.001903,0.000667,-0.000702,-0.000531,-0.000209,-0.000151
3,0.001055,0.001594,0.002304,0.017834,0.001578,0.004021,0.002475,0.003974,0.001742,-0.000630,...,0.001477,-0.000359,0.002038,0.000707,0.002582,0.001175,0.000068,-0.000449,-0.000263,-0.000217
4,0.000955,0.000780,0.000666,0.001578,0.001973,0.001122,0.000945,0.002132,0.001166,0.000823,...,0.000379,0.000979,0.000683,0.001081,0.001049,0.000643,0.000289,0.000560,0.000005,-0.000039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,0.000721,0.000503,0.000667,0.001175,0.000643,0.000806,0.000466,0.000477,-0.000539,0.000201,...,0.000097,0.000271,0.000434,0.000113,0.000835,0.001859,0.000041,0.000072,-0.000035,-0.000020
386,0.000239,0.000168,-0.000702,0.000068,0.000289,-0.000153,0.000127,0.000712,0.000850,0.000472,...,-0.000297,0.000063,-0.000044,-0.000636,-0.000935,0.000041,0.002375,0.001591,0.000298,0.000232
387,0.000470,0.000167,-0.000531,-0.000449,0.000560,-0.000102,0.000220,0.001139,0.000483,0.000435,...,-0.000398,0.000591,0.000282,0.000201,-0.000326,0.000072,0.001591,0.001815,0.000200,0.000174
388,-0.000028,-0.000076,-0.000209,-0.000263,0.000005,-0.000113,0.000044,-0.000188,-0.000038,0.000063,...,0.000012,-0.000018,-0.000053,-0.000070,-0.000305,-0.000035,0.000298,0.000200,0.000119,0.000074


Create optimization model below:

In [9]:
# Define model. 
mod_2 = Model()

# Define decision variables.
w_mod_2 = mod_2.addVars(n) # weights of all 390 stocks

# Constraint 1 - expected monthly return
return_con_mod_2 = mod_2.addConstr(sum(w_mod_2[i] * avg_return_mod_2[i] for i in range(n)) >= 0.005)

# Constraint 2 - sum of weights
weight_con_mod_2 = mod_2.addConstr(sum(w_mod_2[i] for i in range(n)) == 1)

# Constraint 3 - non-negativity constraints
non_negative_mod_2 = {}
for i in range(n):
    non_negative_mod_2[i] = mod_2.addConstr(w_mod_2[i] >= 0.0)

# Create the objective function, and set it to be minimized.
mod_2.setObjective(sum(w_mod_2[i] * w_mod_2[j] * C_mod_2[i, j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

mod_2.update()

mod_2.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0xf0eca57f
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3.90e+05 3.22e-07  1.00e+06     0

**Optimal risk:** 2.87857564e-05 <br>
**Solver time:** 15 iterations and 0.04 seconds

### c) Model 3

Find monthly and average returns for all 390 stocks below:

In [10]:
returns_mod_3 = pd.DataFrame(data = returns)

returns_mod_3.head()

,0,1,2,3,4,5,6,7,8,9,...,380,381,382,383,384,385,386,387,388,389
0,-0.072464,-0.039267,-0.053846,0.022305,-0.014797,-0.055249,-0.001686,-0.131646,0.045455,-0.013986,...,-0.003195,0.012278,-0.005682,-0.102857,-0.011650,0.070946,0.093834,0.045652,0.024668,0.014981
1,0.031250,0.070845,0.018293,0.007273,0.032541,0.140351,0.146959,0.069971,0.095652,-0.014184,...,0.134615,-0.008086,-0.001905,0.019108,0.023576,0.032597,0.078431,0.051975,0.009259,0.002460
2,0.030303,-0.020356,0.025948,-0.061372,-0.033939,0.046154,-0.032401,0.092643,0.007937,0.028777,...,0.042373,0.072011,0.104962,0.006250,0.072937,0.006110,-0.070455,0.009881,-0.009174,-0.003681
3,0.014706,0.000000,0.000000,-0.076923,0.000000,-0.034314,-0.073059,0.004988,-0.055118,0.000000,...,-0.059621,-0.005070,-0.037997,-0.055901,-0.118068,-0.029352,0.061125,0.027397,0.009259,0.003695
4,0.021739,0.031169,0.052529,-0.095833,-0.001255,0.030457,0.064039,-0.007444,0.008333,-0.027972,...,-0.017291,0.021656,0.043088,0.006579,-0.014199,0.007299,-0.004608,0.015238,0.023853,0.012270


In [11]:
avg_return_mod_3 = np.zeros(n)
avg_return_mod_3 = np.mean(returns_mod_3,axis=0)

avg_return_mod_3

0      0.008356
1      0.011798
2      0.012202
3      0.053553
4      0.013364
         ...   
385    0.015104
386    0.008576
387    0.006485
388    0.000247
389   -0.000476
Length: 390, dtype: float64

Find covariance matrix for all 390 stocks below:

In [22]:
C_mod_3 = np.zeros((n,n))
C_mod_3 = np.cov(np.transpose(returns_mod_3))

cov_mod_3 = pd.DataFrame(data = C_mod_3)

cov_mod_3

,0,1,2,3,4,5,6,7,8,9,...,380,381,382,383,384,385,386,387,388,389
0,0.002124,0.001198,0.001128,0.001055,0.000955,0.001135,0.000895,0.002331,0.000414,0.000843,...,0.000710,0.001010,0.001060,0.000791,0.001240,0.000721,0.000239,0.000470,-0.000028,-0.000021
1,0.001198,0.002620,0.001678,0.001594,0.000780,0.001181,0.001134,0.003082,0.000371,0.000887,...,0.000461,0.000692,0.001065,0.000841,0.001935,0.000503,0.000168,0.000167,-0.000076,-0.000060
2,0.001128,0.001678,0.004387,0.002304,0.000666,0.000355,0.000533,0.000991,-0.000377,0.000162,...,0.000577,0.000524,0.000629,0.001823,0.001903,0.000667,-0.000702,-0.000531,-0.000209,-0.000151
3,0.001055,0.001594,0.002304,0.017834,0.001578,0.004021,0.002475,0.003974,0.001742,-0.000630,...,0.001477,-0.000359,0.002038,0.000707,0.002582,0.001175,0.000068,-0.000449,-0.000263,-0.000217
4,0.000955,0.000780,0.000666,0.001578,0.001973,0.001122,0.000945,0.002132,0.001166,0.000823,...,0.000379,0.000979,0.000683,0.001081,0.001049,0.000643,0.000289,0.000560,0.000005,-0.000039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,0.000721,0.000503,0.000667,0.001175,0.000643,0.000806,0.000466,0.000477,-0.000539,0.000201,...,0.000097,0.000271,0.000434,0.000113,0.000835,0.001859,0.000041,0.000072,-0.000035,-0.000020
386,0.000239,0.000168,-0.000702,0.000068,0.000289,-0.000153,0.000127,0.000712,0.000850,0.000472,...,-0.000297,0.000063,-0.000044,-0.000636,-0.000935,0.000041,0.002375,0.001591,0.000298,0.000232
387,0.000470,0.000167,-0.000531,-0.000449,0.000560,-0.000102,0.000220,0.001139,0.000483,0.000435,...,-0.000398,0.000591,0.000282,0.000201,-0.000326,0.000072,0.001591,0.001815,0.000200,0.000174
388,-0.000028,-0.000076,-0.000209,-0.000263,0.000005,-0.000113,0.000044,-0.000188,-0.000038,0.000063,...,0.000012,-0.000018,-0.000053,-0.000070,-0.000305,-0.000035,0.000298,0.000200,0.000119,0.000074


Create optimization model below:

In [52]:
# Define model and paramters.
mod_3 = Model()

max_stocks = 4 # stock limit

# Define decision variables.
w_mod_3 = mod_3.addVars(n) # weights of all 390 stocks
x_mod_3 = mod_3.addVars(n, vtype = GRB.BINARY) # binary variable for the variances of stocks selected

# Constraint 1 - selecting 4 stocks at most
max_con_mod_3 = mod_3.addConstr(sum(x_mod_3[i] for i in range(n)) <= 4)

# Constraint 2 - expected monthly return
return_con_mod_3 = mod_3.addConstr(sum(w_mod_3[i] * avg_return_mod_3[i] for i in range(n)) >= 0.005)

# Constraint 3 - sum of weights
weight_con_mod_3 = mod_3.addConstr(sum(w_mod_3[i] for i in range(n)) == 1)

# Constraint 4 - non-negativity constraints
non_negative_mod_3 = {}
for i in range(n):
    non_negative_mod_3[i] = mod_3.addConstr(w_mod_3[i] >= 0.0)
                                
# Constraint 5 - weights and inclusion of stock
w_con_mod_3 = {}
for i in range(n):
    w_con_mod_3[i] = mod_3.addConstr(w_mod_3[i] <= x_mod_3[i])

# Create the objective function, and set it to be minimized.
mod_3.setObjective(sum(w_mod_3[i] * w_mod_3[j] * C_mod_3[i, j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

mod_3.update()

mod_3.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x27a0aad9
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Found heuristic solution: objective 0.0136011
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

In [45]:
w_opt_mod_3 = [w_mod_3[i].x for i in range(n)]

df_weights_mod_3 = pd.DataFrame(data = w_opt_mod_3, columns=['weights'], index=[tickers])
df_weights_mod_3 = df_weights_mod_3[df_weights_mod_3['weights'] != 0]

df_weights_mod_3

,weights
CME,0.126411
LLY,0.075476
NVDA,0.043754
BND,0.754359


**Optimal risk:** Best objective 6.753470760728e-05, best bound 6.753470760728e-05, gap 0.0000% <br>
**Solver time:** Explored 63026 nodes (2530906 simplex iterations) in 23.33 seconds (46.89 work units) <br>
**Weights:** CME = 0.126411, LLY = 0.075476, NVDA = 0.043754, BND = 0.754359

## Question 2

### a) Model 1 vs. Model 2

Optimal risk in Model 1 is higher than optimal risk in Model 2. This is because we are only given the option to choose from four assets that have been pre-selected for us. Some of these assets could have high volatility and high variances that increase their level of risk.

### b) Model 2 vs. Model 3

Optimal risk in Model 2 is lower than optimal risk in Model 3. This is because we can invest in up to 390 assets in Model 2 while we can only invest in 4 assets at most for Model 3. Having less assets to potentially invest in will increase risk for your selected portfolio. 

## Question 3

### a) Maximum Time Limit

*Note: I know the problem asks us to terminate after 30 seconds, but I set my timer to 20 seconds because my model finds an optimal solution before 30 seconds is reached without any timer set initially.*

In [54]:
# Use Model 3 and set it to terminate after 20 seconds
mod_3.Params.TimeLimit = 20.0

mod_3.setObjective(sum(w_mod_3[i] * w_mod_3[j] * C_mod_3[i, j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

mod_3.update()

mod_3.optimize()

Set parameter TimeLimit to value 20
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x27a0aad9
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]

MIP start from previous solve produced solution with objective 6.75347e-05 (0.03s)
Loaded MIP start from previous solve with objective 6.75347e-05

Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 wo

The objective function value at termination has a lower bound (6.179192164043e-05) compared to the optimal value obtained in 1c (6.753470760728e-05). In addition, there's a gap value of 8.5035% in our new model while the original model in 1c had a 0% gap.

### b) Maximum Acceptable Optimality Gap

In [55]:
# Use Model 3 and set it to terminate after reaching a gap of 10%
mod_3.Params.MIPGap = 0.1

mod_3.setObjective(sum(w_mod_3[i] * w_mod_3[j] * C_mod_3[i, j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

mod_3.update()

mod_3.optimize()

Set parameter MIPGap to value 0.1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x27a0aad9
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]

MIP start from previous solve produced solution with objective 6.75347e-05 (0.03s)
Loaded MIP start from previous solve with objective 6.75347e-05

Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)

Root relaxation: objective 2.878501e-05, 129 iterations, 0.01 seconds (0.01 work

The solver time at termination here has a lower value (17.78 seconds) compared to the solution time obtained in 1c (23.33 seconds). In addition, there are less simplex iterations (2010914 vs. 2530906) and also less work units (38.98 vs. 46.89) compared to the original model.